In [1]:
!wget --header="Host: s3-ap-southeast-1.amazonaws.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,hi;q=0.6" --header="Referer: https://internshala.com/" "https://s3-ap-southeast-1.amazonaws.com/internshala-uploads/chat-uploads/5dc121c2da7871572938178-6949877.zip" -O "Text_Similarity_Dataset.zip" -c

--2019-11-07 04:43:58--  https://s3-ap-southeast-1.amazonaws.com/internshala-uploads/chat-uploads/5dc121c2da7871572938178-6949877.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.106
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6949877 (6.6M) [application/zip]
Saving to: ‘Text_Similarity_Dataset.zip’

Text_Similarity_Dat 100%[===================>]   6.63M  16.8MB/s    in 0.4s    

2019-11-07 04:43:59 (16.8 MB/s) - ‘Text_Similarity_Dataset.zip’ saved [6949877/6949877]



In [0]:
import zipfile
zip_ref = zipfile.ZipFile("Text_Similarity_Dataset.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [0]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("Text_Similarity_Dataset.csv")
data.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


In [5]:
data.tail()

,Unique_ID,text1,text2
4018,4018,labour plans maternity pay rise maternity pay ...,no seasonal lift for house market a swathe of ...
4019,4019,high fuel costs hit us airlines two of the lar...,new media battle for bafta awards the bbc lead...
4020,4020,britons growing digitally obese gadget lover...,film star fox behind theatre bid leading actor...
4021,4021,holmes is hit by hamstring injury kelly holmes...,tsunami to hit sri lanka banks sri lanka s b...
4022,4022,nuclear dumpsite plan attacked plans to allow...,x factor show gets second series tv talent sho...


In [0]:
!pip3 install --quiet "tensorflow>=1.7"
!pip3 install --quiet tensorflow-hub

![alt text](https://www.gstatic.com/aihub/tfhub/universal-sentence-encoder/example-similarity.png)

**Ref : https://tfhub.dev/google/universal-sentence-encoder/2**

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2), axis=1)
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, 0.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)

In [0]:
text_a = data['text1'].tolist()
text_b = data['text2'].tolist()

def run_sts_benchmark(session):
  """Returns the similarity scores"""
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={
          sts_input1: text_a,
          sts_input2: text_b
      })
  return scores


with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  scores = run_sts_benchmark(session)

In [11]:
len(scores)

4023

In [0]:
data["Similarity_Score"] = scores

In [13]:
data.head()

,Unique_ID,text1,text2,Similarity_Score
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...,-0.463501
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...,-0.194790
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...,-0.002321
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...,-0.130618
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...,-0.147261


In [17]:
data[data["Similarity_Score"]>=0.4].shape

(18, 4)

In [0]:
def partition(x):
    if x < 0.4:
        return 1
    return 0

In [0]:
data["Label"] = data["Similarity_Score"].apply(partition)

In [28]:
data.head()

,Unique_ID,text1,text2,Similarity_Score,Label
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...,-0.463501,1
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...,-0.194790,1
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...,-0.002321,1
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...,-0.130618,1
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...,-0.147261,1


In [29]:
data.tail()

,Unique_ID,text1,text2,Similarity_Score,Label
4018,4018,labour plans maternity pay rise maternity pay ...,no seasonal lift for house market a swathe of ...,-0.154795,1
4019,4019,high fuel costs hit us airlines two of the lar...,new media battle for bafta awards the bbc lead...,-0.339178,1
4020,4020,britons growing digitally obese gadget lover...,film star fox behind theatre bid leading actor...,-0.278862,1
4021,4021,holmes is hit by hamstring injury kelly holmes...,tsunami to hit sri lanka banks sri lanka s b...,-0.384129,1
4022,4022,nuclear dumpsite plan attacked plans to allow...,x factor show gets second series tv talent sho...,-0.312227,1


In [30]:
data["Label"].value_counts()

1    4005
0      18
Name: Label, dtype: int64

In [0]:
submission = data.drop(["text1","text2","Label"],axis=1)
submission.to_csv("result.csv",index=False)